In [ ]:
import torch

import copy
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from torch import nn, optim

import torch.nn.functional as F

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/DACON/train_features.csv')

In [ ]:
target = pd.read_csv('/content/drive/MyDrive/DACON/train_labels.csv')
train_labels = target

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/DACON/test_features.csv')
submission = pd.read_csv('/content/drive/MyDrive/DACON/sample_submission.csv')

In [ ]:
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [ ]:
train_mean = train.copy()
train_max = train.copy()
train_min = train.copy()

In [ ]:
# train_ 데이터프레임에 group 컬럼을 추가한다.

train_mean['group'] = 1
train_max['group'] = 1
train_min['group'] = 1

In [ ]:
for i in range(59 , -1 , -1):
  train_mean.loc[(train_.time < (i+1)*10), 'group'] = i
  train_max.loc[(train_.time < (i+1)*10), 'group'] = i
  train_min.loc[(train_.time < (i+1)*10), 'group'] = i
# 0 ~ 9
# 10 ~ 19
# 20 ~ 29
# 30 ~ 39
# ...
# 590 ~ 599

In [ ]:
group_train_mean = train_mean.groupby(['id', 'group'])
group_train_max = train_max.groupby(['id', 'group'])
group_train_min = train_min.groupby(['id' , 'group'])

In [ ]:
df_mean =  pd.DataFrame(group_train_mean.mean())
df_max = pd.DataFrame(group_train_max.max())
df_min = pd.DataFrame(group_train_min.min())

In [ ]:
df_1 = pd.merge(left = df_mean , right = df_max , on=['id' , 'group'])

In [ ]:
df_2 = pd.merge(left = df_1 , right = df_min , on =['id', 'group'])

In [ ]:
df_finish = df_2.drop(['time_x' , 'time_y' , 'time'], axis = 1)

In [ ]:
def preprocess(data):
  train_mean_ = data.copy()
  train_max_ = data.copy()
  train_min_ = data.copy()

  train_mean_['group'] = 1
  train_max_['group'] = 1
  train_min_['group'] = 1

  for i in range(59 , -1 , -1):
    train_mean_.loc[(train_.time < (i+1)*10), 'group'] = i
    train_max_.loc[(train_.time < (i+1)*10), 'group'] = i
    train_min_.loc[(train_.time < (i+1)*10), 'group'] = i
  
  group_train_mean = train_mean_.groupby(['id', 'group'])
  group_train_max = train_max_.groupby(['id', 'group'])
  group_train_min = train_min_.groupby(['id' , 'group'])

  mean_data =  pd.DataFrame(group_train_mean.mean())
  max_data = pd.DataFrame(group_train_max.max())
  min_data = pd.DataFrame(group_train_min.min())

  df_1 = pd.merge(left = mean_data , right = max_data , on=['id' , 'group'])
  df_2 = pd.merge(left = df_1 , right = min_data , on =['id', 'group'])

  df_finish_preprocess = df_2.drop(['time_x' , 'time_y' , 'time'], axis = 1)

  return df_finish_preprocess

In [ ]:
df_finish.shape

(187500, 18)

In [ ]:
X = torch.FloatTensor(df_finish.values).view(-1, 60, 18)

In [ ]:
X.shape

torch.Size([3125, 60, 18])

In [ ]:
train_labels

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [ ]:
y = torch.FloatTensor(train_labels['label'].values)

In [ ]:
y.shape

torch.Size([3125])

In [ ]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [ ]:
y = y.view(-1, 1)

In [ ]:
y.shape

torch.Size([3125, 1])

In [ ]:
class Net(torch.nn.Module):
  def __init__(self , input_size , hidden_size):
    super(Net, self).__init__()
    self.lstm = torch.nn.LSTM(input_size , hidden_size , batch_first= True)
    self.fc1 = torch.nn.Linear(hidden_size , 128 , bias = True)
    self.relu = torch.nn.ReLU()
    self.fc2 = torch.nn.Linear(128,61 ,bias = True)

  def forward(self, x):
    x, _status = self.lstm(x)
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

In [ ]:
output = Net(18,61)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(output.parameters(), 1e-3)

In [ ]:
y = y.squeeze(1).long()

In [ ]:
for i in range(400):
  optimizer.zero_grad()
  outputs = output(X)
  loss = criterion(outputs[:,-1,:] , y)
  loss.backward()
  optimizer.step()
  
  train_acc = torch.sum(torch.argmax(outputs[:,-1,:], axis = 1) == y)
  
  print('epoch : ',i, " loss : ", loss.item())
  print('acc : ', (train_acc / outputs.shape[0]).numpy())

In [ ]:
df_test = preprocess(test)

In [ ]:
df_test.shape

(46920, 18)

In [ ]:
test_tensor = torch.FloatTensor(df_test.values)

In [ ]:
test_tensor = test_tensor.view(-1,60,18)

In [ ]:
print(test_tensor.shape)

torch.Size([782, 60, 18])


In [ ]:
with torch.no_grad():
  #X_test = torch.FloatTensor(test.iloc[:,2:].values).view(-1, 600, 6)
  pred_test = output(test_tensor)

In [ ]:
pred_test.shape

torch.Size([782, 60, 61])

In [ ]:
x_numpy = pred_test[:, -1, :]

In [ ]:
print(x_numpy.shape)

torch.Size([782, 61])


In [ ]:
#x_numpy = np.argmax(x_numpy, axis = 1)

In [ ]:
#x_numpy

array([37, 26,  3, ..., 15, 26, 55])

In [ ]:
#y_numpy = y.numpy()
#y_numpy

array([37, 26,  3, ..., 15, 26,  2])

In [ ]:
import sklearn.metrics as metrics

In [ ]:
acc = metrics.accuracy_score(x_numpy , y_numpy)
print('acc : ', acc)

acc :  0.97664


In [ ]:
test_df = pd.DataFrame(x_numpy.numpy())

In [ ]:
test_df.to_excel('/content/drive/MyDrive/DACON/excel_test.xlsx의 사본.xlsx')

In [ ]:
torch.save(output.state_dict(), '/content/drive/MyDrive/DACON/data/EMS_LSTM_epoch_400_mean_min_max')

In [ ]:
X.shape

torch.Size([3125, 60, 18])

In [ ]:
with torch.no_grad():
  #X_test = torch.FloatTensor(test.iloc[:,2:].values).view(-1, 600, 6)
  pred_test = output(X)

In [ ]:
argmax_train = torch.argmax(pred_test[:,-1,:], axis= 1)

In [ ]:
x_numpy = argmax_train.numpy()

In [ ]:
x_numpy

array([37, 26,  3, ..., 15, 26, 55])

In [ ]:
y_numpy

array([37, 26,  3, ..., 15, 26,  2])

In [ ]:
# T 에서 0인것만 모은다. (인덱스 반환)
zero_idx = np.where(x_numpy == 0)
#(array([ 118,  141,  181, 1515, 1628, 2249, 2324, 2517, 2519, 2586, 3101]),)

# T 에서 0인것만 모은다.

zero_t = x_numpy[np.where(x_numpy == 0)]
zero_y = y_numpy[zero_idx]
print(zero_idx)
print('zero_t : ',zero_t)
print('zero_y : ',zero_y)
# print('acc when label is  ',i, ": ", acc)

(array([ 118,  141,  181, 1515, 1628, 2249, 2324, 2517, 2519, 2586, 3101]),)
zero_t :  [0 0 0 0 0 0 0 0 0 0 0]
zero_y :  [0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
import sklearn.metrics as metric

In [ ]:
for i in range(60):

  idx = np.where(x_numpy == i)

  t = x_numpy[np.where(x_numpy == i)]
  y = y_numpy[idx]
  acc = metric.accuracy_score(y, t)
  print('acc when label is  ',i, ": ", acc * 100)

acc when label is   0 :  100.0
acc when label is   1 :  100.0
acc when label is   2 :  100.0
acc when label is   3 :  100.0
acc when label is   4 :  100.0
acc when label is   5 :  100.0
acc when label is   6 :  100.0
acc when label is   7 :  100.0
acc when label is   8 :  98.96907216494846
acc when label is   9 :  89.74358974358975
acc when label is   10 :  85.71428571428571
acc when label is   11 :  95.23809523809523
acc when label is   12 :  91.66666666666666
acc when label is   13 :  100.0
acc when label is   14 :  100.0
acc when label is   15 :  100.0
acc when label is   16 :  100.0
acc when label is   17 :  100.0
acc when label is   18 :  100.0
acc when label is   19 :  100.0
acc when label is   20 :  100.0
acc when label is   21 :  100.0
acc when label is   22 :  100.0
acc when label is   23 :  100.0
acc when label is   24 :  91.42857142857143
acc when label is   25 :  100.0
acc when label is   26 :  97.80503550677857
acc when label is   27 :  97.14285714285714
acc when label is 

In [ ]:
arg = [9, 10 , 51, 52, 58]
for i,u in enumerate (arg):
  # T 에서 0인것만 모은다. (인덱스 반환)
  label_idx = u
  zero_idx = np.where(x_numpy == label_idx)
  #(array([ 118,  141,  181, 1515, 1628, 2249, 2324, 2517, 2519, 2586, 3101]),)

  # T 에서 0인것만 모은다.

  zero_t = x_numpy[np.where(x_numpy == label_idx)]
  zero_y = y_numpy[zero_idx]
  print('when i is :', u)
  print('zero_t : ',zero_t)
  print('zero_y : ',zero_y)
  print('=============================================================================')
  # print('acc when label is  ',i, ": ", acc)

when i is : 9
zero_t :  [9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9
 9 9]
zero_y :  [ 9  9  9  9  9  9  9  9  9  9  9 24  9  9  9  9  9  9  9 30  9  9  9  9
  9  9  9  9  9  9  9 31  9  9  9  9 58  9  9]
when i is : 10
zero_t :  [10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10]
zero_y :  [30 10 10 10 10 10 10 10 10 10 10 10 10 14 33 10 10 10 10 10 10]
when i is : 51
zero_t :  [51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51
 51 51 51]
zero_y :  [51 51 51 52 51 51 15 51 51 51 51 51 11 51 51 51 51 51 51 51 11 51 11 51
 51 51 51]
when i is : 52
zero_t :  [52 52 52 52 52 52 52 52 52 52 52 52]
zero_y :  [52 52 28 52 52 52 52 51 52 12 52 52]
when i is : 58
zero_t :  [58 58 58 58 58 58 58 58 58 58 58 58 58 58 58 58 58 58 58 58 58]
zero_y :  [58 58 58 24 58 58 58 24 58 58 58 58 58 58 58 31 58 58 58 58 58]


In [ ]:
df_test

acc_x_x   acc_y_x   acc_z_x  ...       gy_x       gy_y        gy_z
id   group                                ...                                  
3125 0     -0.442843 -0.053869 -0.263157  ... -23.759140  18.748804   -3.605706
     1     -0.955634 -0.008549 -0.344066  ... -23.466845 -19.816078   -1.483997
     2     -0.831829  0.016192 -0.328560  ... -16.010805 -44.335938  -11.625415
     3     -0.813808 -0.031349 -0.121235  ... -27.668966 -46.604496  -26.785818
     4     -1.128031 -0.099283 -0.015475  ... -23.657070  -7.729901  -18.966589
...              ...       ...       ...  ...        ...        ...         ...
3906 55    -0.230866 -0.685296  0.641896  ...  -8.157926 -69.022030    8.471539
     56    -0.089739 -0.692458  0.631095  ... -18.180579  -8.227655  -31.036427
     57     0.070699 -0.734292  0.574833  ...  -6.206841  19.405557  -67.384476
     58    -0.115750 -0.733167  0.630194  ... -37.703669 -27.579613  -69.475716
     59     0.084417 -0.818748  0.676649  ... -29.900725 -36.586219 -190.358982

[46920 rows x 18 columns]